# FER with CNN on optical flows

## Imports

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import h5py
import cv2
from sklearn.metrics import zero_one_loss
from sklearn.metrics import accuracy_score

#Pytorch learning
import torch
from torchsummary import summary
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
import torch.optim as optim
import torch.nn.functional as F

from exp_rec import CNN as ExpNet

#Functions import
from functions import get_imflow
from functions import mag_ang_std_mean
from functions import write_results

## Data loading

In [2]:
#######################################################################
# Loading data tables from H5py files
#######################################################################

h5f = h5py.File('train/data_train_images.h5','r')
train_images = h5f['data_train_images'][:]
h5f.close()

h5f = h5py.File('test/data_test_images.h5','r')
test_images = h5f['data_test_images'][:]
h5f.close()

h5f = h5py.File('train/data_train_labels.h5','r')
train_labels = h5f['data_train_labels'][:]
train_labels = train_labels
h5f.close()

In [3]:
train_images.shape

(540, 10, 300, 200)

In [4]:
get_imflow(test_images[0][0],test_images[0][9]).shape

(2, 300, 200)

In [5]:
test_flows = []
for i in range(len(test_images)):
    test_flows.append(get_imflow(test_images[i][0],test_images[i][9]))

In [6]:
train_flows = []
for i in range(len(train_labels)):
    train_flows.append(get_imflow(train_images[i][0],train_images[i][9]))

In [7]:
mean_mags, std_mags, mean_angs, std_angs = mag_ang_std_mean(train_flows)

In [8]:
train_normalized_flows = []
for flow in train_flows:
    train_normalized_flows.append(np.array([(flow[0]-mean_mags)/std_mags,(flow[1]-mean_angs)/std_angs]))

In [42]:
mean_mags, std_mags, mean_angs, std_angs = mag_ang_std_mean(train_flows)

In [43]:
test_normalized_flows = []
for flow in test_flows:
    test_normalized_flows.append(np.array([(flow[0]-mean_mags)/std_mags,(flow[1]-mean_angs)/std_angs]))

In [10]:
trainset = {}
train_labels = torch.Tensor(train_labels).long()
for i in range(len(train_labels)):
    trainset[i] = (train_normalized_flows[i],train_labels[i])

## Model architecture

In [11]:
batch_size = 16

In [12]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

In [13]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [99]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Downsampling
        self.down = nn.Sequential(
            nn.Conv2d(2, 8, 3, padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2)
            )
        # Upsampling
        self.up = nn.Sequential(
            nn.Linear(29600,256),
            nn.ReLU(True),
            nn.Linear(256,6),
            nn.Softmax(dim=1)
            )
        

    def forward(self, img):
        out = self.down(img)
        out = self.up(out.view(out.size(0), -1))
        return out

## Training

In [100]:
cnn = CNN()

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)

In [40]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data[0]
        labels = data[1]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 4:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[1,     5] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[1,    10] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[1,    15] loss: 0.004
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[1,    20] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[1,    25] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16,

torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[8,     5] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[8,    10] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[8,    15] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[8,    20] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[8,    25] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[8,    30] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300

[15,     5] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[15,    10] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[15,    15] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[15,    20] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[15,    25] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[15,    30] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([12, 

torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[22,    10] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[22,    15] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[22,    20] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[22,    25] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[22,    30] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([12, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size

torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[29,    10] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[29,    15] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[29,    20] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[29,    25] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[29,    30] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([12, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size

[36,    10] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[36,    15] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[36,    20] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[36,    25] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[36,    30] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([12, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[37,     5

torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[43,    15] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[43,    20] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[43,    25] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[43,    30] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([12, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[44,     5] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size

torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[50,    15] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[50,    20] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[50,    25] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
[50,    30] loss: 0.003
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([16, 2, 300, 200])
torch.Size([12, 2, 300, 200])
Finished Training


In [41]:
torch.save(cnn.state_dict(), "cnn_normalized_flows.pth")

## Test model

In [15]:
cnn = CNN()
cnn.load_state_dict(torch.load("cnn_normalized_flows.pth"))

<All keys matched successfully>

In [16]:
train_img = torch.Tensor(train_normalized_flows)

In [17]:
outputs = cnn(train_img)
_, predicted = torch.max(outputs, 1)

In [19]:
accuracy_score(train_labels, predicted)

1.0

In [44]:
test_img = torch.Tensor(test_normalized_flows)

In [45]:
outputs = cnn(test_img)
_, predicted = torch.max(outputs, 1)

In [46]:
len(predicted)

131

In [32]:
write_results(predicted,"results_normalized_flows.csv")

In [53]:
df = pd.read_csv("results_cnn_2.csv")
pred0 = df['expression'].values

In [56]:
df = pd.read_csv("results_3.csv")
pred1 = df['expression'].values

In [57]:
accuracy_score(pred1,pred0)

0.42748091603053434

## Fine tuning

In [103]:
class CNN_PreTrained(nn.Module):
    def __init__(self):
        super(CNN_PreTrained, self).__init__()

        # Upsampling
        self.down = nn.Sequential(
            nn.Conv2d(2, 8, 3, padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2)
            )
        # Upsampling
        self.up = nn.Sequential(
            nn.Linear(8*8*32,256),
            nn.ReLU(True),
            nn.Linear(256,6),
            nn.Softmax()
            )

    def forward(self, img):
        out = self.down(img)
        out = self.up(out.view(out.size(0), -1))
        return out

In [104]:
cnn_pre_trained = CNN_PreTrained()
cnn_pre_trained.load_state_dict(torch.load("model_exp/cnn1.pth",map_location=torch.device('cpu')))

<All keys matched successfully>

In [108]:
for param in cnn_pre_trained.parameters():
    param.requires_grad = False

In [109]:
cnn_pre_trained.up[0] = nn.Linear(29600,256)

In [112]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data[0]
        labels = data[1]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = cnn_pre_trained(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 4:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,     5] loss: 0.004
[1,    10] loss: 0.004
[1,    15] loss: 0.004
[1,    20] loss: 0.004
[1,    25] loss: 0.004
[1,    30] loss: 0.004
[2,     5] loss: 0.004
[2,    10] loss: 0.004
[2,    15] loss: 0.004
[2,    20] loss: 0.004
[2,    25] loss: 0.004
[2,    30] loss: 0.004
[3,     5] loss: 0.004
[3,    10] loss: 0.004
[3,    15] loss: 0.004
[3,    20] loss: 0.004
[3,    25] loss: 0.004
[3,    30] loss: 0.004
[4,     5] loss: 0.004
[4,    10] loss: 0.004
[4,    15] loss: 0.004
[4,    20] loss: 0.004
[4,    25] loss: 0.004
[4,    30] loss: 0.004
[5,     5] loss: 0.004
[5,    10] loss: 0.004
[5,    15] loss: 0.004
[5,    20] loss: 0.004
[5,    25] loss: 0.004
[5,    30] loss: 0.004
[6,     5] loss: 0.004
[6,    10] loss: 0.004
[6,    15] loss: 0.004
[6,    20] loss: 0.004
[6,    25] loss: 0.004
[6,    30] loss: 0.004
[7,     5] loss: 0.004
[7,    10] loss: 0.004
[7,    15] loss: 0.004
[7,    20] loss: 0.004
[7,    25] loss: 0.004
[7,    30] loss: 0.004
[8,     5] loss: 0.004
[8,    10] 